# Summary

This notebook contains attempts to create the Earth's magnetic field model and calculate the magnetic declination.

The first method was to use the law of cosines. Expand the law of cosines to the triangle on a sphere, and use it to calculate the magnetic declination. The triangle is created from three points, the true north, the magnetic north, and an observer. Move the observer around the earth and see how the magnetic declination varies. Draw contour line on a world map to visualize declination.

The second method was to think of the Earth's magnetic field as created from the two magnetic monopoles. Locate each monopole under the magnetic north pole and south pole. Then calculate the magnetic field on a given point on the sphere. Using vector operations, calculate the horizontal component of the magnetic field and magnetic declination. Draw contour line and compare it with method 1.

Finally, compare both results with the WMM model, and analyze the results and errors.

## Expanding the law of consines

Assume a sphere with the radius of 1. A, B, and C are three different points on the sphere and a, b, and c are their opposite sides accordingly.
Then, the following equation holds for all triangle ABC. ( $\angle$ signs are omitted for simplicity.)

$$
\cos a = \sin b \sin c \cos A + \cos b \cos c \\[0.5em]
or, \\[0.5em]
\cos A = {\cos a - \cos b \cos c \over \sin b \sin c}
$$

Thus, if we know the lenghts of three sides of a triangle on a sphere, we can then calculate angles with the formula.

## Calculating magnetic declination

Given three points, the true north, the magnetic north, and the observer, we can think of a triangle on a sphere with those points as vertices. Assuming that the magnetic declination is the same as the angle between two sides of a triangle shareing the observer point, we can calculate the magnetic declination with the above formula. 

Let's say that the true north is point A, the magnetic north is point B, and the observer is point C. Then we can get three side lengths.

$$
A = | \phi_m - \phi_o | \\[0.5em]
b = {\pi \over 2} - \theta_o \\[0.5em]
c = {\pi \over 2} - \theta_m \\[0.5em]
\cos a = \sin b \sin c \cos A + \cos b \cos c \\[0.7em]
\sin a = \sqrt{1 - \cos^2 a} \\[0.5em]
\cos C = {\cos c - \cos a \cos b \over \sin b \sin c}
$$

where $\theta_m$ and $\phi_m$ are latitude and longitude of the magnetic north, and $\theta_o$ and $\phi_o$ are those of the observer.

In [1]:
import numpy as np

feature_x = np.arange(-180, 180, 5)
feature_y = np.arange(-90, 90, 5)

# Magnetic pole location was from WMM pole locations (year 2023)
# https://www.ngdc.noaa.gov/geomag/data/poles/WMM2020_NP.xy
magnetic_pole_lat = 86.146
magnetic_pole_lon = 146.826

# Observer longitude and latitude
X, Y = np.meshgrid(feature_x, feature_y)

A = np.abs(magnetic_pole_lon - X) * np.pi / 180
b = (90 - Y) * np.pi / 180
c = (90 - magnetic_pole_lat) * np.pi / 180
cos_a = np.sin(b) * np.sin(c) * np.cos(A) + np.cos(b) * np.cos(c)
sin_a = np.sqrt(1 - np.square(cos_a))
cos_C = np.clip((np.cos(c) - cos_a * np.cos(b)) / (sin_a * np.sin(b)), -1, 1)

# Magnetic declination
Z = np.arccos(cos_C)


In [2]:

import plotly.graph_objects as go

fig = go.Figure(
    data=[
        go.Contour(
            x=feature_x,
            y=feature_y,
            z=Z,
            contours=dict(coloring="fill"),
            showlegend=False,
            ncontours=20,
        ),
    ]
)

fig.show()

## Discussion

Earth magnetic field is a complex model. In this (over) simplified model, we are missing too many things, and therefore not getting the suitable result. 
Followings are the potential reasons for the error:

 - Magnetic declination is actually based on the magnetic field's horizontal component, not the shortest path from the observer to the magnetic pole. 
 - We are not considering the magnetic south pole. This matters because the Earth's magnetic field is not symmetrical in reality.
 - The Earth is not a perfect sphere.
 